In [29]:
# Setup if running in colab
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
  try:
    if runonce:
      print("Already ran")
  
  except:
    runonce = True
    !pip install wandb
    !git clone https://github.com/Jimmy-Nnilsson/StudieGrupp3_MLProjekt.git
    
    import wandb
    wandb.login()

In [ ]:
import tqdm
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import load_img, img_to_array

import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

True

In [ ]:
# Get base project directory
if not RunningInCOLAB:
#   project_path = Path(os.getcwd()).parent.parent

  for i, p in enumerate(Path(os.getcwd()).parts):
    if p == "StudieGrupp3_MLProjekt":
        break
    pathparts = list(Path(os.getcwd()).parts[0:i+2])
    project_path = Path(pathparts[0],"\\\\".join(pathparts[1:]))
else:
  project_path = Path('/content/StudieGrupp3_MLProjekt/')
datapath = (project_path /'data/processed/')

CLASSES = {0 : 'yes', 1 : 'no'}
# Loops through pathlist and reads and resizes images
def read_image(pathlist : list, size : int)-> list:
    data = []
    for path in pathlist:
        image=load_img(path, color_mode='rgb', target_size=(size, size))
        # image=load_img(path, color_mode='rgb', target_size=(size, size))
        image=img_to_array(image)
        # image=image/255.0
        data.append(image)
    data = np.asarray(data, dtype=np.uint8)
    return data

# Makes input and label data from folder locations.
# Loops through location "subfolder/CLASSES"
def get_sets(subfolder : str, CLASSES : dict, size : int):
    folder_paths = []
    folder_labels = []
    labels = []
    for k,v in CLASSES.items():
        # input datapath generation
        folder_paths += list((datapath / f"2_split_{v}/{subfolder}").rglob("*"))
    # Label data generation
    folder_labels = [0 if x.stem.split('_')[1] == 'yes' else 1 for x in folder_paths]
    folder_labels = np.asarray(folder_labels, dtype=np.uint8)
    # Extract images from datapaths
    img_list = read_image(folder_paths, size)

    return img_list, folder_labels

def get_training_set(CLASSES : dict, size : int):
    folder_paths = []
    folder_labels = []
    labels = []
    for k,v in CLASSES.items():
        # input datapath generation
        folder_paths += list((datapath / f"3_aug_{v}_train/").rglob("*"))
        # folder_paths += list((datapath / f"3_augmentation_train/3_aug_geo_{v}_train/").rglob("*"))
        # folder_paths += list((datapath / f"3_augmentation_train/3_aug_pix_{v}_train/").rglob("*"))
        # print(folder_paths)
    # Label data generation
    folder_labels = [0 if x.stem.split('_')[1] == 'yes' else 1 for x in folder_paths]
    # Extract images from datapaths
    img_list = read_image(folder_paths, size)

    return img_list, folder_labels

In [ ]:
# Dataset inspect
# Read images to variables
size = 224
X_aug_train, y_aug_train = get_training_set(CLASSES, size)
X_train, y_train = get_sets('train', CLASSES, size)
X_val, y_val = get_sets('val', CLASSES, size)
X_test, y_test = get_sets('test', CLASSES, size)

In [ ]:
# Mind model processing
# Finetune not complete
configs = dict(
    project_name = "s2", #Project Name
    mode = 'run', #{'offline', 'run', 'disabled', 'dryrun', 'online'} # WandB run status
    job_type = "", #Run type for WandB
    group = "", # Group in WandB
    sub_group = "_aug_geo_pix",

    class_names = CLASSES, # Classes for training
    training_set = (keras.applications.vgg19.preprocess_input(X_aug_train), y_aug_train),

    image_width = X_train[0].shape[0], # Picture width for model input
    image_height = X_train[0].shape[1], # Picture height for model input
    image_channels = X_train[0].shape[2], # Picture channels for model input

    pretrain_weights = 'imagenet', # pretrained weights for basemodel if any
    batch_size = 4, # Batchsize for training
    init_learning_rate = 0.001, # Initial training rate if no callback is used
    lr_decay_rate = 0.1, #decayrate of training rate
    epochs = 50, # Epochs to train
    optimizer = 'adam', # The optimizer used by the ml model
    loss_fn = 'binary_crossentropy', # Loss function
    metrics = ['accuracy'], # Metrics
    earlystopping_patience = 5, # For the early stopping callback

    dataset = "Brain_MRI_Images_for_Brain_Tumor_Detection",
    fine_tune_learning_rate = 1e-5, # learningrate Used during fine tuning
    fine_tune_epochs = 10, # Epochs ran at finetuning

    architecture = "",# To be defined f"{base_model._name.upper()} global_average_pooling2d",
    model_name = '' # set after model is defined # Name of the ml Model

)

In [ ]:
sweep_config = {
    'method': "random",
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize',
    },
    'parameters': {
        "optimizer": {
            "values": ['adam', 'sgd', 'rmsprop']
        },
        "nodes": {
            "values": [128, 256, 512]
        },
        "epochs": {
            "values": [1,2,3,5]
        },
        "learning_rate": {
            "distribution": "uniform",
            "min": 0.0001,
            "max": 0.1
        },
        "batch_size": {
            "distribution": "q_log_uniform",
            "q": 1,
            "min": 32,
            "max": 128
        },
    },
}

In [ ]:
sweep_config = {
  'method': 'random', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
        "optimizer": {
            "values": ['adam', 'sgd', 'rmsprop']
        },
      'batch_size': {
          'values': [32, 64, 128]
      },
      'learning_rate':{
          'values': [0.01, 0.005, 0.001, 0.0005, 0.0001]
      },
      'model_bridge':{
          'values': ['flatten', 'gap']
      },
      'layers':{
          'values': [0, 1, 2, 3]
      },
      'nodes':{
          'values': [32, 64, 128]
      }
  }
}

In [ ]:
def Model(bridge, layers, nodes):
    kwarg = dict(weights=configs['pretrain_weights'], include_top=False, input_shape=(configs['image_width'],configs['image_height'],configs['image_channels']))
    base_model = keras.applications.vgg19.VGG19(**kwarg)

    base_model.trainable = False
    # bridge between basemodel and endstuff
    if bridge == 'gap':
        x = keras.layers.GlobalAveragePooling2D()(base_model.output)
    elif bridge == 'flatten':
        x = keras.layers.GlobalAveragePooling2D()(base_model.output)

    if layers > 0:
        for i in range(layers):
            x = keras.layers.Dense(int(nodes/(i+1)), activation='relu')(x)

    outputs = keras.layers.Dense(1, activation='sigmoid')(x)
    configs['group'] = f'{base_model._name}{configs["sub_group"]}'
    configs['architecture'] = base_model._name
    model  = keras.Model(base_model.input, outputs, name=f'Baseline_{base_model._name.upper()}')

    return model

    
def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, logits)

    return loss_value

    
def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)

    return loss_value

In [ ]:
def train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=10, 
          log_step=200, 
          val_log_step=50):
  
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in tqdm.tqdm(enumerate(train_dataset), total=len(train_dataset)):
            loss_value = train_step(x_batch_train, y_batch_train, 
                                    model, optimizer, 
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value = test_step(x_batch_val, y_batch_val, 
                                       model, loss_fn, 
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))

        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_states()
        val_acc_metric.reset_states()

        # 3️⃣ log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': float(train_acc), 
                   'val_loss': np.mean(val_loss),
                   'val_acc':float(val_acc)})

In [ ]:
def sweep_train(config_defaults=None):
    # Set default values
    config_defaults = {
        "batch_size": 64,
        "learning_rate": 0.01
    }
    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)  # this gets over-written in the Sweep

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 2
    wandb.config.log_step = 20
    wandb.config.val_log_step = 50
    wandb.config.architecture_name = configs['model_name']
    wandb.config.dataset_name = configs['dataset']

    # build input pipeline using tf.data
    train_dataset = tf.data.Dataset.from_tensor_slices(configs['training_set'])
    train_dataset = (train_dataset.shuffle(buffer_size=512)
                                  .batch(wandb.config.batch_size)
                                  .prefetch(buffer_size=tf.data.AUTOTUNE))

    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
    val_dataset = (val_dataset.batch(wandb.config.batch_size)
                              .prefetch(buffer_size=tf.data.AUTOTUNE))

    # initialize model
    model = Model(wandb.config.model_bridge, wandb.config.layers, wandb.config.nodes)

    # Instantiate an optimizer to train the model.
    if wandb.config.optimizer == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=wandb.config.learning_rate)
    elif wandb.config.optimizer == "rmsprop":
        optimizer = keras.optimizers.RMSprop(learning_rate=wandb.config.learning_rate)
    elif wandb.config.optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=wandb.config.learning_rate)

    # Instantiate a loss function.
    loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

    # Prepare the metrics.
    train_acc_metric = keras.metrics.BinaryAccuracy()
    val_acc_metric = keras.metrics.BinaryAccuracy()

    train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=wandb.config.epochs, 
          log_step=wandb.config.log_step, 
          val_log_step=wandb.config.val_log_step)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project=configs['project_name'])


Create sweep with ID: bs8wai5f
Sweep URL: https://wandb.ai/bex_team/s2/sweeps/bs8wai5f


In [ ]:
# wandb.agent(sweep_id, function=sweep_train, count=10)

In [ ]:
# sweep_id = wandb.sweep(sweep_config, project=configs['project_name'])
sweep_id = "pc0g0ncm"


# wandb.agent(sweep_id, function=sweep_train, count=10)

wandb.agent(
    sweep_id, function=sweep_train, entity="bex_team", project=configs['project_name'], count=1
)

wandb: Agent Starting Run: 5qiczmgq with config:
wandb: 	batch_size: 32
wandb: 	layers: 3
wandb: 	learning_rate: 0.0005
wandb: 	model_bridge: flatten
wandb: 	nodes: 32
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.



Start of epoch 0


100%|██████████| 63/63 [00:40<00:00,  1.56it/s]


Training acc over epoch: 0.6260
Validation acc: 0.3750

Start of epoch 1


100%|██████████| 63/63 [00:38<00:00,  1.64it/s]


Training acc over epoch: 0.6295
Validation acc: 0.3750


acc,▁█
epochs,▁█
loss,█▁
val_acc,▁▁
val_loss,▁█
acc,0.62946
epochs,1
loss,0.69317
val_acc,0.375
val_loss,0.69294
